# Cat vs. Dog (Binary Classification) with N11 Model 

## Background
Binary classification in the context of images, is the process of teaching a model to categorize a given image into one of two possible classes. 
We need to classify images to automate tasks that require visual understanding, enabling computers to interpret and make decisions based on the content of a picture.
This ability to automatically categorize images unlocks countless applications across various industries, from healthcare to transportation.

- Classify a skin lesion as either benign or malignant (cancerous) from a photograph, assisting dermatologists in early cancer detection. 
- An autonomous vehicle needs to classify objects as pedestrians or other objects.
- Classify different types of crops from an aerial photo, assess plant health by identifying diseased vs. healthy crops.
- Unlock your smartphone with your face is a process to identify authorized personnel vs. unauthorized intruders.

## Objectives:
- Practice neural network model with a hidden layer.
- Review binary classification.
- Get started using [PyTorch](https://pytorch.org/).

 <font color=582c83> 

## Exercises:
1. (5%) Data Preprocessing
2. (15%) Exercise 2: N11 Model Construction
3. (10%) Exercise 3: Model Assessment
4. (20%) Exercise 4: Gradients Computation
5. (40%) Exercise 5: Model Optimization
6. (6%) Exercise 6: New Images Test
7. (4%) Exercise 7: Model Analyses

</font>



## 1. Dataset Preparation with PyTorch

### 1.1 Create Annotation Files
Data is not always stored in NumPy arrays. Most cases, you will have to organize and annotate the raw data stored in your hard drive. For image data, you want it to be organized as the following way.
``` console
root/dog/xxx.jpg
root/dog/xxy.jpg
root/dog/xxz.jpg

root/cat/123.jpg
root/cat/456.jpg
root/cat/789.jpg
```
To grab image information and store them in an comma-seperated values (CSV) file:
1. Visit the data directory, grab all images' paths and corresponding categories.
2. Save the paths and categories of images in an `.csv` file 

In [ ]:
import os
from glob import glob
import pandas as pd
from pathlib import Path

# Locate train and validation directories
root_dir = "./dataset"  # locate dataset directory from this repo in the whole system
train_cat_dir = Path(root_dir) / "training" / "cats"
train_dog_dir = Path(root_dir) / "training" / "dogs"
val_cat_dir = Path(root_dir) / "validation" / "cats"
val_dog_dir = Path(root_dir) / "validation" / "dogs"

# Glob training files
train_cat_files = list(train_cat_dir.glob("**/*.jpg"))
train_dog_files = list(train_dog_dir.glob("**/*.jpg"))
print(f"There are {len(train_cat_files)} cat images, and {len(train_dog_files)} dog images in the training dataset")
df_train = pd.DataFrame({
    'path': list(train_cat_files) + list(train_dog_files),
    'label': ['cat'] * len(list(train_cat_files)) + ['dog'] * len(list(train_dog_files))
})
df_train.to_csv('annotations_train.csv', header=False, index=False)  

# Glob validation files
val_cat_files = list(val_cat_dir.glob("**/*.jpg"))
val_dog_files = list(val_dog_dir.glob("**/*.jpg"))
print(f"There are {len(val_cat_files)} cat images, and {len(val_dog_files)} dog images in the validation dataset")
df_val = pd.DataFrame({
    'path': list(val_cat_files) + list(val_dog_files),
    'label': ['cat'] * len(list(val_cat_files)) + ['dog'] * len(list(val_dog_files))
})
df_val.to_csv('annotations_val.csv', header=False, index=False)  


### 1.2 Manage Datasets using [PyTorch](https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files)
1. Inherit the `Dataset` class to build a customized `CatDogDataset` class.
2. Instantiate the customized class to a `dataset_train` and `dataset_test` .
3. Further create dataloaders to shuffle the data and access the full matrix of the features and the targets. 

In [ ]:
from torchvision.io import decode_image, ImageReadMode
from torchvision.transforms.v2 import Resize
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(3321)

class CatDogDataset(Dataset):
    def __init__(self, annotations_file):
        self.imgs_info = pd.read_csv(annotations_file, header=None)

    def __len__(self):
        return len(self.imgs_info)
    
    def __getitem__(self, idx):
        img_path = self.imgs_info.iloc[idx, 0]
        raw_img = decode_image(img_path, mode=ImageReadMode.RGB)
        # resize image to (64, 64)
        image = Resize((64, 64))(raw_img)
        category = 1. if self.imgs_info.iloc[idx, 1] == 'dog' else 0.
        sample = {'image': image, 'category': category}

        return sample
    


### 1.3 Preprocess the Data
A typical binary classification dataset is made up with a feature matrix: $\mathbf{X} = [^{(1)}\mathbf{x}, ^{(2)}\mathbf{x}, ..., ^{(M)}\mathbf{x}]^T$. and a target vector $\mathbf{y} = [^{(1)}y, ^{(2)}y, ..., ^{(M)}y]^T$. Where $M$ is the total number of instances in the dataset, $^{(m)}\mathbf{x}$ is a normalized and flattened image array, and $^{(m)}y \in \{0, 1\}$.

- A colored image is usually represented by a 3-dimensional array with shape **`(width, height, 3)`** or **`(3, width, height)`**.
- When a digital image is loaded, each pixel bears an integer value ranged **0~255** to represent the color intensity.

![](https://miro.medium.com/v2/format:webp/1*pFywKuWmz7Xk07OXxPiX2Q.png)

We will access the raw data by extracting it from the dataloaders. Then, process and prepare the raw data so that it can be used in later steps.


In [ ]:
# Extract features/images and targets/labels
dataset_train = CatDogDataset(annotations_file='annotations_train.csv')
dataset_val = CatDogDataset(annotations_file='annotations_val.csv')

# Create data loaders
dataloader_train = DataLoader(dataset_train, batch_size=1000, shuffle=True)  # large batch size to get all data in one batch
dataloader_val = DataLoader(dataset_val, batch_size=1000, shuffle=False)
# Extract all data from dataloaders
raw_data_train = next(iter(dataloader_train))
raw_data_val = next(iter(dataloader_val))

# Separate features from targets 
raw_features_train = raw_data_train['image'].numpy()
raw_labels_train = raw_data_train['category'].numpy()
raw_features_val = raw_data_val['image'].numpy()
raw_labels_val = raw_data_val['category'].numpy()
# Uncomment 4 lines below to exam the raw data
# print(f"Raw training features shape: {raw_features_train.shape}, raw trainging labels shape: {raw_labels_train.shape}")
# print(f"Raw validation features shape: {raw_features_val.shape}, raw validation labels shape: {raw_labels_val.shape}")
# print(f"A sample of raw features array:\n {raw_features_train[0]}")
# print(f"First 10 raw training labels: {raw_labels_train[:10]}")

# Visualize a few sample images
fig, axs = plt.subplots(1, 4)
for i in range(4):
    sample_img = raw_features_train[i]
    sample_cls = raw_labels_train[i]
    axs[i] = plt.subplot(1, 4, i + 1)
    axs[i].set_title(f'{"dog" if sample_cls == 1. else "cat"}')
    axs[i].axis('off')
    axs[i].imshow(np.transpose(sample_img, (1, 2, 0)))  # image is already in (H, W, C) format
    plt.tight_layout()

# Rerun this cell to get different samples.

### <font color=582C83>(5%) Exercise 1: Data Preprocessing</font>
1. Rescale pixel color intensity values within range: `[0, 1]` using float numbers.
2. Reshape feature array to `(# samples, # pixels)`
3. Reshape label array to `(# samples, 1)`.


In [ ]:


### START CODE HERE ### (≈ 4 lines of code)
features_train = None
labels_train = None
features_val = None
labels_val = None
### END CODE HERE ###

# Sanity check
print(f"Processed training features shape: {features_train.shape}, a sample:\n {features_train[0]}")
print(f"Processed training labels shape: {labels_train.shape}")
print(f"Processed validation features shape: {features_val.shape}, a sample:\n {features_val[0]}")
print(f"Processed validation labels shape: {labels_val.shape}")


**Expected Output**:
>
```console
Processed training features shape: (557, 12288)
Processed training labels shape: (557, 1)
Processed validation features shape: (140, 12288)
Processed validation labels shape: (140, 1)
```

## 2. Multi-Input Single-Hidden Layer Single-Output Model
$$
\begin{align*}
    \hat{\mathbf{y}} &= \sigma(\mathbf{X}^{[1]} \cdot \mathbf{w}^{[2]T} + b^{[2]}) \\
                     &= \sigma(\sigma(\mathbf{X}^{[0]} \cdot \mathbf{W}^{[1]T} + \mathbf{b}^{[1]}) \cdot \mathbf{w}^{[2]T} + b^{[2]})
\end{align*}
$$

where $\mathbf{X}^{[0]}$ is the input features matrix, $\mathbf{X}^{[1]}$ is the hidden features matrix, and $\hat{\mathbf{y}}$ is model predictions matrix. 
The model is governed by 2 layers of parameters: $\mathbf{W}^{[1]}$, $\mathbf{b}^{[1]}$, $\mathbf{w}^{[2]}$, and $b^{[2]}$. 
$\sigma(\cdot)$ is the [sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function).
Please refer to lecture [slides](https://linzhanguca.github.io/_docs/deep_learning-2025/0929/nnN11.pdf) for the shape of each matrix/array.

### <font color=#582C83>(15%) Exercise 2: N11 Model Construction </font> 
1. Let the linear model: $\mathbf{Z} = \mathbf{X} \cdot \mathbf{w}^T + b$ to take feature matrix $\mathbf{X}$ as the input and output a transformed/intermediate feature matrix $\mathbf{Z}$.
2. Apply sigmoid function on $\mathbf{Z}$, so that the prediction will be: $\mathbf{\hat{y}} = \sigma(\mathbf{Z}) = 1 / (1 + e^{-\mathbf{Z}})$ 

In [ ]:
def sigmoid(in_features):
    """ Sigmoid function
    Args:
        in_features: independent variable, could be an arrary of any shape or a scalar. 
    Returns:
        activated_features: dependent variable, could be an arrary of any shape or a scalar. 
    """
    ### START CODE HERE ### (≈ 1 line of code)
    activated_features = None
    ### END CODE HERE ###
    return activated_features

def linear(in_features, weights, biases):
    """ Linear function
    Args:
        in_features: input feature matrix, 2d array with shape (# samples, # input features)
        weights: weight parameter matrix, 2d array with shape (# next layer features , # input features)
        biases: bias parameter vector, 2d array with shape (1, # next layer features)
    Returns:
        linear_output: linear model output feature matrix, 2d array with shape (# samples, # next layer features)
    """
    ### START CODE HERE ### (≈ 1 lines of code)
    linear_output = None
    ### END CODE HERE ###
    return linear_output

def forward(in_features, params):
    """ Forward function
    Args:
        in_features: feature matrix, 2d array with shape (# samples, # pixels)
        params: a dictionary of weights and biases
    Returns:
        predictions: predicted probabilities, a column vector or 2d array with shape (# samples, 1)
    """

    ### START CODE HERE ### (≈ 2 lines of code)
    features_1 = None
    predictions = None
    ### END CODE HERE ###
    return predictions, features_1


# Sanity check
params_dummy = {
    'W1': np.random.normal(size=(5, features_train.shape[1])),
    'b1': np.random.normal(size=(1, 5)),
    'W2': np.random.normal(size=(1, 5)),
    'b2': np.random.normal()
}
preds_dummy, X1_dummy = forward(features_train, params_dummy)
print(f"Hidden layer (X1) shape: {X1_dummy.shape}")
print(f"Hidden layer (X1) samples:\n{X1_dummy[:3]}")
print(f"Shape of dummy predictions: {preds_dummy.shape}")
print(f"Dummy prediction samples:\n{preds_dummy[:3]}")
print(f"Dummy predicted classes samples:\n{(preds_dummy[:3] >= 0.5).astype(int)}")


**Expected Output**:
>
```console
Hidden layer (X1) shape: (557, 5)
Hidden layer (X1) samples:
[[1.00000000e+00 1.00000000e+00 3.21777360e-19 9.24947905e-02
  1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 8.60858067e-30 5.80196383e-42
  1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.71898486e-20 4.23640927e-26
  1.00000000e+00]]
Shape of dummy predictions: (557, 1)
Dummy prediction samples:
[[0.47896982]
 [0.49083586]
 [0.49083586]]
Dummy predicted classes samples:
[[0]
 [0]
 [0]]
```

## 3. Evaluate Model

### 3.1. Binary Cross Entropy Loss
$$\mathcal{L}(\mathbf{\hat{y}}, \mathbf{y}) = \frac{1}{M} \Sigma [-\mathbf{y} \log \hat{\mathbf{y}} - (1 - \mathbf{y}) \log(1 - \hat{\mathbf{y}})]$$

### 3.2. Accuracy
$$Acc = \frac{TP + TN}{TP + TN + FP + FN}$$

### <font color=#582c83>(10%) Exercise 3: Model Assessment </font>


In [ ]:
def bce_loss(predictions, labels):
    """
    Compute the Binary Cross Entropy (BCE) loss.
    
    Args:
        predictions: model predicted values, a 2d array with shape (# samples, 1)
        labels: labeled values from data set, a 2d array with shape (# samples, 1)
    Returns:
        loss_value: averaged BCE error, a scalar
    """
    ### START CODE HERE ### (≈ 1 line)
    loss_value = None
    ### END CODE HERE ###
    return loss_value
    
def accuracy(predictions, labels):
    """
    Compute the accuracy of predictions.
    
    Args:
        predictions: model predicted values, a 2d array with shape (# samples, 1)
        labels: labeled values from data set, a 2d array with shape (# samples, 1)
    Returns:
        acc: accuracy, a scalar
    """
    ### START CODING HERE ### (≈ 2 lines)
    predicted_classes = None
    acc = None
    return acc
    ### END CODING HERE ###

# Sanity check
print(f"Dummy model BCE loss: {bce_loss(preds_dummy, labels_train)}")
print(f"Dummy model accuracy: {accuracy(preds_dummy, labels_train) * 100}%")

**Expected Output**:
>
```console
Dummy model BCE loss: 0.7176122919341839
Dummy model accuracy: 48.473967684021545%
```


## 4. Back-Propagate Gradients of Loss
Compute gradients of loss so that we can update model parameters later.
$$
\nabla \mathcal{L} = 
\begin{bmatrix} 
    \frac{\partial \mathcal{L}}{\partial \mathbf{W}^{[1]}} &\frac{\partial \mathcal{L}}{\partial \mathbf{b}^{[1]}} &\frac{\partial \mathcal{L}}{\partial \mathbf{w}^{[2]}} &\frac{\partial \mathcal{L}}{\partial b^{[2]}} 
\end{bmatrix} 
$$

The computation of the gradients starts from the last layer, then back-propagate to the first layer. 
$$
\begin{align*}
    \frac{\partial \mathcal{L}}{\partial \mathbf{w}^{[2]}} &= \frac{\partial \mathcal{L}}{\partial \hat{\mathbf{y}}} \frac{\partial \hat{\mathbf{y}}}{\partial \mathbf{Z}^{[2]}} \frac{\partial \mathbf{Z}^{[2]}}{\partial \mathbf{w}^{[2]}} = \frac{1}{M} (\hat{\mathbf{y}} - \mathbf{y})^T \cdot \mathbf{X}^{[1]} \\
    \frac{\partial \mathcal{L}}{\partial b^{[2]}} &= \frac{\partial \mathcal{L}}{\partial \hat{\mathbf{y}}} \frac{\partial \hat{\mathbf{y}}}{\partial \mathbf{Z}^{[2]}} \frac{\partial \mathbf{Z}^{[2]}}{\partial b^{[2]}} = \overline{(\hat{\mathbf{y}} - \mathbf{y})} \\
    \frac{\partial \mathcal{L}}{\partial \mathbf{X}^{[1]}} &= \frac{\partial \mathcal{L}}{\partial \hat{\mathbf{y}}} \frac{\partial \hat{\mathbf{y}}}{\partial \mathbf{Z}^{[2]}} \frac{\partial \mathbf{Z}^{[2]}}{\partial \mathbf{X}^{[1]}} = (\hat{\mathbf{y}} - \mathbf{y}) \cdot \mathbf{w}^{[2]} \\
    \frac{\partial \mathcal{L}}{\partial \mathbf{W}^{[1]}} &= \frac{\partial \mathcal{L}}{\partial \mathbf{X}^{[1]}} \frac{\partial \mathbf{X}^{[1]}}{\partial \mathbf{Z}^{[1]}} \frac{\partial \mathbf{Z}^{[1]}}{\partial \mathbf{W}^{[1]}} = \frac{1}{M} [(\hat{\mathbf{y}} - \mathbf{y}) \cdot \mathbf{w}^{[2]} * \mathbf{X}^{[1]} * (1 - \mathbf{X}^{[1]})]^T \cdot \mathbf{X}^{[0]} \\
    \frac{\partial \mathcal{L}}{\partial \mathbf{b}^{[1]}} &= \frac{\partial \mathcal{L}}{\partial \mathbf{X}^{[1]}} \frac{\partial \mathbf{X}^{[1]}}{\partial \mathbf{Z}^{[1]}} \frac{\partial \mathbf{Z}^{[1]}}{\partial \mathbf{b}^{[1]}} = \overline{(\hat{\mathbf{y}} - \mathbf{y}) \cdot \mathbf{w}^{[2]} * \mathbf{X}^{[1]} * (1 - \mathbf{X}^{[1]})}, axis=0, keepdims
\end{align*}
$$

### <font color=#582c83>(20%) Exercise 4: Gradients Computation</font>



In [ ]:
def backward(in_features, features_1, predictions, labels, params):
    """ Gradient function with sigmoid activation
    Args:
        in_features: feature matrix, a 2d array with shape (# samples, # pixels)
        features_1: hidden layer feature matrix, a 2d array with shape (# samples, # hidden features)
        predictions: model predicted value, a 2d array with shape (# samples, 1)
        labels: labeled value from data set, a 2d array with shape (# samples, 1)
        params: a dictionary of weights and biases
    Returns:
        grads: a dictionary of gradients of weights and biases
    """

    ### START CODE HERE ### (≈ 6 lines)
    dL_dW2 = None
    dL_db2 = None
    dL_dX1 = None
    dL_dW1 = None
    dL_db1 = None
    grads = {
        'dW2': dL_dW2,
        'db2': dL_db2,
        'dW1': dL_dW1,
        'db1': dL_db1
    }   

    return grads
    ### END CODE HERE ### 

grads_dummy = backward(features_train, X1_dummy, preds_dummy, labels_train, params_dummy)
for k, v in grads_dummy.items():
    print(f"{k}: {v}") 
    print(f"Shape of {k}: {v.shape}")   

**Expected Output**:
>
```console
dW2: [[-0.01219751 -0.01291893  0.02279078 -0.00305472 -0.00633631]]
Shape of dW2: (1, 5)
db2: -0.011168723623745864
Shape of db2: ()
dW1: [[-1.83808861e-06 -3.60190685e-07 -4.61408613e-07 ... -5.76820151e-05
  -6.07048124e-05 -6.22402946e-05]
 [-1.77579955e-04 -1.80499871e-04 -1.83843446e-04 ... -1.22578799e-04
  -1.09420635e-04 -1.09276882e-04]
 [ 3.72156964e-04  4.05010601e-04  4.25819595e-04 ...  4.01500770e-04
   3.62029743e-04  3.52201928e-04]
 [ 7.81283167e-06  2.18910660e-05  2.50437615e-05 ... -9.66289643e-06
  -1.37238647e-05  2.59703713e-06]
 [-4.44899727e-05 -4.90495903e-05 -5.15890191e-05 ... -2.62571885e-05
  -2.86883308e-05 -2.76296182e-05]]
Shape of dW1: (5, 12288)
db1: [[-4.70166025e-05 -5.89589280e-04  1.29078047e-03  1.93598262e-04
  -1.57950863e-05]]
Shape of db1: (1, 5)
```

## 5. Update Model Parameters
By tweaking the model parameters along the gradient a small step (learning rate, $\alpha$) iteratively, we expect to bring the model loss down to a reasonable scale.

- Initialize parameters
- Repeat until converge
$$
\begin{align*}
    \mathbf{W}^{[1]} &= \mathbf{W}^{[1]} - \alpha \frac{\partial \mathcal{L}}{\partial \mathbf{W}^{[1]}} \\
    \mathbf{b}^{[1]} &= \mathbf{b}^{[1]} - \alpha \frac{\partial \mathcal{L}}{\partial \mathbf{b}^{[1]}} \\
    \mathbf{w}^{[2]} &= \mathbf{w}^{[2]} - \alpha \frac{\partial \mathcal{L}}{\partial \mathbf{w}^{[1]}} \\
    b^{[2]} &= b^{[2]} - \alpha \frac{\partial \mathcal{L}}{\partial b^{[2]}}
\end{align*}   
$$
### <font color=582c83>(40%) Exercise 5: Model Optimization</font>
1. Define a function to compute gradient of loss
2. Perform gradient descent optimization using appropriate iterations and learning rate.
    1. Initialize weights and bias
    2. Make predictions
    3. Log training loss and test loss
    4. Update weights and bias
    5. Repeat 2 to 5 until converge.
    
<font color=red> **Bring training (dataset) accuracy over 70%** </font>

In [ ]:
### START CODE HERE ### (≈ 17 lines)

# Initialize parameters
params = {
    'W1': None,
    'b1': None,
    'W2': None,
    'b2': None,
}

# Set hyperparameters
num_iters = None
learning_rate = None

# Reserve metrics storage
losses_train, lossese_val = [], []
accuracies_train, accuracies_val = [], []

# Optimization iterations
for i in range(num_iters):
    # Forward pass
    preds_train, features_1_train = None
    preds_val, features_1_val = None
    # Compute loss and accuracy
    loss_train = None
    loss_val = None
    acc_train = None
    acc_val = None
    losses_train.append(loss_train)
    lossese_val.append(loss_val)
    accuracies_train.append(acc_train)
    accuracies_val.append(acc_val)
    # Log progress (every 50 iterations)   
    if i % 100 == 0 or i == num_iters - 1:
        print(f"Iter {i}: Train Loss {loss_train}, Train Acc {acc_train}")
        print(f"Iter {i}: Validation Loss {loss_val}, Validation Acc {acc_val}")
    # Back-propagation
    grads = None
    # Gradient descent parameter update
    params['W1'] = None
    params['b1'] = None
    params['W2'] = None
    params['b2'] = None

### END CODE HERE ### 


### Visualize Training and Model Performance


In [ ]:

# Plot loss and accuracy curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(num_iters), losses_train, range(num_iters), lossese_val)
plt.legend(['Train', 'Validation'])
plt.title("Loss Decrease")
plt.xlabel("Iteration")
plt.ylabel("BCE Loss")
plt.subplot(1, 2, 2)
plt.plot(range(num_iters), accuracies_train, range(num_iters), accuracies_val)
plt.legend(['Train', 'Validation'])
plt.title("Accuracy Growth")
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.ylim(0, 1)

# Confusion Matrix for Test Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm_train = confusion_matrix(labels_train, (preds_train >= 0.5).astype(int))
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=["cat", "dog"])
disp_train.plot(cmap='PuRd')
plt.title('Confusion Matrix - Training Data')
cm_val = confusion_matrix(labels_val, (preds_val >= 0.5).astype(int))
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=["cat", "dog"])
disp_test.plot(cmap='PuRd')
plt.title('Confusion Matrix - Validation Data')

## 6. Test and Analyze
There are 6 pictures of (3) dogs and (3) cats in this repository. 
Test these new images using the model you've just trained.
Hopefully, all the cats say "Meow" and dogs say "Woof".

### <font color=582c83> (6%) Exercise 6: New Images Test </font>



In [ ]:
test_dir = "./test_images"  # locate dataset directory from this repo in the whole system
test_cat_dir = Path(test_dir) / "cats"
test_dog_dir = Path(test_dir) / "dogs"


# Glob test files
test_cat_files = list(test_cat_dir.glob("**/*"))
test_dog_files = list(test_dog_dir.glob("**/*"))
test_files = test_cat_files + test_dog_files

for test_file in test_files:
    image_tensor = decode_image(None, mode=ImageReadMode.RGB)  # decode image file to pytorch tensor
    image = Resize((None, None))(image_tensor)  # resize image to correct size
    image_np = None  # convert image tensor to numpy array
    image_flatten = None  # rescale and reshape
    dog_likelihood, _ = None  # use model to predict
    is_dog = None  # use correct threshold to classify
    if is_dog.squeeze():
        print(f"{test_file.name}: Woof!") 
    else:
        print(f"{test_file.name}: Meow!")



### <font color=582c83> (4%) Exercise 7: Model Analyses </font>

Was your model performing similar on training data, validation data and test pictures? If not, what could be the cause?
> Please write your answer below.

# Congratulations! You have finished this assignment!